Wildfire Neural Network

Combo of tutorials: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/ & https://www.tensorflow.org/tutorials/load_data/csv

In [127]:
#importing libraries used

import keras.models 
from keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn import model_selection
import os


In [104]:
#setting my working directory to the desired location
os.chdir('/Users/petri142/Wildfire_proj')

In [8]:
#creating a function to download a zip and unzip it in my desired location
def download_data(dw_url,destination):
    import wget
    import zipfile
    
    filename = wget.download(dw_url)
    print("Successfully downloaded", filename)
    
    with zipfile.ZipFile(destination + "/" + filename, 'r') as zip_ref:
        zip_ref.extractall(destination)

In [9]:
#using my function to downloand and unzip my data
download_data('https://github.com/ouladsayadyounes/WildFires/archive/refs/heads/master.zip','/Users/petri142/Wildfire_proj')

Successfully downloaded WildFires-master.zip


In [105]:
#opening the CSV and viewing part of it
wildfires = pd.read_csv('WildFires-master/WildFires_DataSet.csv', float_precision='high')
wildfires.head()

,NDVI,LST,BURNED_AREA,CLASS
0,0.506782,14584.272727,4.692308,no_fire
1,0.522150,14655.833333,5.000000,no_fire
2,0.682284,14780.000000,5.000000,fire
3,0.120046,13298.500000,3.500000,no_fire
4,0.568734,14743.000000,5.000000,no_fire


Let's get out data in tip top shape. Data normalization.
Part of the reason why a NN can fail is when very large numbers skew the model creating "constant" behavior of the NN for predictions. Ran into an issue where the model was predicting identical classes for everything. 

In [106]:
#ML does not like classes that are a string
#change the classes to numbers
wildfires['CLASS'] = wildfires['CLASS'].map({'fire': 1, 'no_fire': 0})

In [107]:
#seperating the features from the labels
wildfire_features = wildfires.copy()
wildfire_labels = wildfire_features.pop('CLASS')
wildfire_labels.tail()

1708    0
1709    0
1710    1
1711    0
1712    1
Name: CLASS, dtype: int64

In [108]:
#create functions to normalize the data for the Neural Net
def normalize(x):
    x_med = x.median()
    x_std = x.std()
    x = ((x-x_med)/x_std)
    return x

def normalize_dataframe(pandas_df):
    pandas_df = pandas_df.apply(normalize)
    
    return pandas_df

In [110]:
wildfire_features = normalize_dataframe(wildfire_features) 
wildfire_features.head()
#now you might be thinking to yourself, "That's shome pretty shweet data"

,NDVI,LST,BURNED_AREA
0,-0.628978,-0.167546,-0.391111
1,-0.463460,0.027480,0.135948
2,1.261242,0.365876,0.135948
3,-4.794266,-3.671703,-2.433466
4,0.038267,0.265038,0.135948


In [111]:
#changing this from a series to values
wildfire_labels = wildfire_labels.values
wildfire_labels

array([0, 0, 1, ..., 1, 0, 1])

In [112]:
wildfire_features = np.array(wildfire_features)
wildfire_features

array([[-6.28978449e-01, -1.67545939e-01, -3.91111091e-01],
       [-4.63459505e-01,  2.74804235e-02,  1.35947705e-01],
       [ 1.26124186e+00,  3.65875733e-01,  1.35947705e-01],
       ...,
       [ 5.15028005e-01,  2.88203790e-01, -3.28993790e+00],
       [ 5.30871209e-01, -2.82003661e+00,  7.16010091e-04],
       [-1.68922379e-01, -1.02464888e+00,  1.35947705e-01]])

In [135]:
#splitting my features and labels into train and test
wf_features_train, wf_features_test, wf_class_train, wf_class_test = model_selection.train_test_split(wildfire_features, wildfire_labels, test_size=0.3)

In [136]:
#defining my model
model = Sequential()
model.add(Dense(12,input_shape=(3,),activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [137]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [138]:
model.fit(wf_features_train, wf_class_train, epochs=300, shuffle=True, batch_size=3)

Epoch 1/300
400/400 [==============================] - 1s 844us/step - loss: 0.5715 - accuracy: 0.7339
Epoch 2/300
400/400 [==============================] - 0s 882us/step - loss: 0.4779 - accuracy: 0.7840
Epoch 3/300
400/400 [==============================] - 0s 876us/step - loss: 0.4678 - accuracy: 0.7840
Epoch 4/300
400/400 [==============================] - 0s 876us/step - loss: 0.4633 - accuracy: 0.7882
Epoch 5/300
400/400 [==============================] - 0s 866us/step - loss: 0.4593 - accuracy: 0.7907
Epoch 6/300
400/400 [==============================] - 0s 872us/step - loss: 0.4581 - accuracy: 0.7898
Epoch 7/300
400/400 [==============================] - 0s 859us/step - loss: 0.4562 - accuracy: 0.7890
Epoch 8/300
400/400 [==============================] - 0s 893us/step - loss: 0.4546 - accuracy: 0.7965
Epoch 9/300
400/400 [==============================] - 0s 859us/step - loss: 0.4543 - accuracy: 0.7948
Epoch 10/300
400/400 [==============================] - 0s 859us/step - l

Make Predictions

In [139]:
score = model.evaluate(wf_features_test, wf_class_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')


Test loss: 0.47700604796409607 / Test accuracy: 0.7879377603530884


In [140]:
y_pred = model.predict(wf_features_test)
y_pred[:10], wf_class_test[:10]

(array([[0.04534352],
        [0.27755713],
        [0.76779175],
        [0.40266702],
        [0.01916772],
        [0.40576324],
        [0.01329795],
        [0.57656664],
        [0.20647982],
        [0.14681703]], dtype=float32),
 array([1, 0, 0, 0, 0, 1, 0, 1, 1, 0]))

In [141]:
for i in y_pred:
    if i > .5:
        print("Fire")
    else:
        print("No fire")


No fire
No fire
Fire
No fire
No fire
No fire
No fire
Fire
No fire
No fire
No fire
No fire
No fire
No fire
Fire
No fire
No fire
No fire
No fire
No fire
Fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
Fire
No fire
Fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
Fire
No fire
No fire
Fire
No fire
No fire
No fire
No fire
Fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
Fire
No fire
No fire
Fire
No fire
No fire
No fire
No fire
No fire
Fire
No fire
No fire
No fire
Fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
Fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No fire
No

In [142]:
#save the trained model
model.save('Wildfire_NN_model/Wildfire_NN_model_success_wNormalization.h5')

In [129]:
#load the saved model
reconstructed_model = keras.models.load_model('Wildfire_NN_model/Wildfire_NN_model.h5')
reconstructed_model.fit(wf_features_test, wf_class_test, epochs=10, batch_size=2)

Epoch 1/10
172/172 [==============================] - 0s 867us/step - loss: 0.5802 - accuracy: 0.7784
Epoch 2/10
172/172 [==============================] - 0s 841us/step - loss: 0.5227 - accuracy: 0.7813
Epoch 3/10
172/172 [==============================] - 0s 859us/step - loss: 0.5048 - accuracy: 0.7843
Epoch 4/10
172/172 [==============================] - 0s 857us/step - loss: 0.4944 - accuracy: 0.7843
Epoch 5/10
172/172 [==============================] - 0s 864us/step - loss: 0.4881 - accuracy: 0.7843
Epoch 6/10
172/172 [==============================] - 0s 872us/step - loss: 0.4813 - accuracy: 0.7843
Epoch 7/10
172/172 [==============================] - 0s 847us/step - loss: 0.4777 - accuracy: 0.7872
Epoch 8/10
172/172 [==============================] - 0s 886us/step - loss: 0.4726 - accuracy: 0.7843
Epoch 9/10
172/172 [==============================] - 0s 871us/step - loss: 0.4702 - accuracy: 0.7872
Epoch 10/10
172/172 [==============================] - 0s 877us/step - loss: 0.467

In [130]:
y1 = reconstructed_model.evaluate(wf_features_test,wf_class_test)
y2 = model.evaluate(wf_features_test,wf_class_test)
print(y1, y2)

11/11 [==============================] - 0s 766us/step - loss: 0.4919 - accuracy: 0.7522
[0.4628147780895233, 0.7871720194816589] [0.491909384727478, 0.7521865963935852]


In [131]:
results = np.testing.assert_allclose(model.predict(wf_features_test), reconstructed_model.predict(wf_features_test))
print(results)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 343 / 343 (100%)
Max absolute difference: 0.49592152
Max relative difference: 2.2367468
 x: array([[1.798490e-01],
       [7.160014e-02],
       [9.795399e-01],...
 y: array([[0.245368],
       [0.196016],
       [0.582867],...